### Libraries

In [2]:
import pandas as pd
import numpy as np
import warnings
import os
warnings.filterwarnings('ignore')

### Load Data

In [3]:
#tmp = pd.read_csv(os.path.join('MachineLearningCVE/', os.listdir('MachineLearningCVE')[3]))
#tmp[' Label'].unique()
tmp1 = pd.read_csv(os.path.join('MachineLearningCVE/', os.listdir('MachineLearningCVE')[0]))
tmp2 = pd.read_csv(os.path.join('MachineLearningCVE/', os.listdir('MachineLearningCVE')[1]))
tmp3 = pd.read_csv(os.path.join('MachineLearningCVE/', os.listdir('MachineLearningCVE')[2]))
tmp4 = pd.read_csv(os.path.join('MachineLearningCVE/', os.listdir('MachineLearningCVE')[3]))
tmp5 = pd.read_csv(os.path.join('MachineLearningCVE/', os.listdir('MachineLearningCVE')[4]))
tmp6 = pd.read_csv(os.path.join('MachineLearningCVE/', os.listdir('MachineLearningCVE')[5]))
tmp7 = pd.read_csv(os.path.join('MachineLearningCVE/', os.listdir('MachineLearningCVE')[6]))
tmp8 = pd.read_csv(os.path.join('MachineLearningCVE/', os.listdir('MachineLearningCVE')[7]))
display(tmp1[' Label'].value_counts())
display(tmp2[' Label'].value_counts())
display(tmp3[' Label'].value_counts())
display(tmp4[' Label'].value_counts())
display(tmp5[' Label'].value_counts())
display(tmp6[' Label'].value_counts())
display(tmp7[' Label'].value_counts())
display(tmp8[' Label'].value_counts())

DDoS      128027
BENIGN     97718
Name:  Label, dtype: int64

PortScan    158930
BENIGN      127537
Name:  Label, dtype: int64

BENIGN    189067
Bot         1966
Name:  Label, dtype: int64

BENIGN    529918
Name:  Label, dtype: int64

BENIGN          288566
Infiltration        36
Name:  Label, dtype: int64

BENIGN                        168186
Web Attack � Brute Force        1507
Web Attack � XSS                 652
Web Attack � Sql Injection        21
Name:  Label, dtype: int64

BENIGN         432074
FTP-Patator      7938
SSH-Patator      5897
Name:  Label, dtype: int64

BENIGN              440031
DoS Hulk            231073
DoS GoldenEye        10293
DoS slowloris         5796
DoS Slowhttptest      5499
Heartbleed              11
Name:  Label, dtype: int64

In [4]:
# load raw data
# 월요일데이터('Monday-WorkingHours.pcap_ISCX.csv')는 정상상태만 존재하므로 데이터의 balance를 위해 제외함
# 화요일 / 수요일 데이터도 제외
files = os.listdir('MachineLearningCVE')
files.remove('Monday-WorkingHours.pcap_ISCX.csv')
files.remove('Tuesday-WorkingHours.pcap_ISCX.csv')
files.remove('Wednesday-workingHours.pcap_ISCX.csv')
display(files)

pd.set_option('display.max_columns', None) # set option to display all rows & columns
pd.set_option('display.max_rows', None)
nRowsRead = None
df_raw = pd.DataFrame()

for file in files:
    tmp_df = pd.read_csv(os.path.join('MachineLearningCVE/', file))
    df_raw = pd.concat([df_raw,tmp_df])
    del tmp_df

['Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
 'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
 'Friday-WorkingHours-Morning.pcap_ISCX.csv',
 'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
 'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv']

In [5]:
# 데이터 용량 간소화 및 연산속도 증가를 위해 데이터타입 축소 (int64 > int32, float64 > float32)
for column in df_raw.columns:
    if df_raw[column].dtype == np.int64:
        maxVal = df_raw[column].max()
        if maxVal < 120:
            df_raw[column] = df_raw[column].astype(np.int8)
        elif maxVal < 32767:
            df_raw[column] = df_raw[column].astype(np.int16)
        else:
            df_raw[column] = df_raw[column].astype(np.int32)
            
    if df_raw[column].dtype == np.float64:
        maxVal = df_raw[column].max()
        minVal = df_raw[df_raw[column]>0][column]
        if maxVal < 120 and minVal>0.01 :
            df_raw[column] = df_raw[column].astype(np.float16)
        else:
            df_raw[column] = df_raw[column].astype(np.float32)

In [6]:
df_raw[' Label'].value_counts()

BENIGN                        871074
PortScan                      158930
DDoS                          128027
Bot                             1966
Web Attack � Brute Force        1507
Web Attack � XSS                 652
Infiltration                      36
Web Attack � Sql Injection        21
Name:  Label, dtype: int64

In [7]:
## 데이터클리닝
# merge web attacks
df_raw[' Label'] = df_raw[' Label'].replace(["Web Attack � Brute Force","Web Attack � XSS","Web Attack � Sql Injection"],"Web Attack")
df_raw[' Label'].value_counts()

# drop missing vaules
print('shape of raw data:                               ',df_raw.shape)
print('missing data(null, NaN, Inf, -Inf):              ',df_raw.isna().sum().sum()) 
df_raw = df_raw[~df_raw.isin([np.nan,np.inf,-np.inf]).any(1)]
print('shape of raw data w/o missing values:            ',df_raw.shape)
      
# drop columns with single unique values 
sing_vals = []
for i in df_raw.columns:
    if df_raw[i].value_counts().nunique() < 2:
        sing_vals.append(i)      

df_raw.drop(sing_vals,axis=1,inplace=True)
df_raw.columns =  df_raw.columns.str.strip()
print('shape of raw data w/o single unique values:      ',df_raw.shape)

# drop duplicate rows
df_raw = df_raw.drop_duplicates(keep='first')
df_raw.reset_index(drop=True,inplace=True)
print('shape of raw data w/o duplicates:                ',df_raw.shape)
 
# print attacks out of total
nRow, nCol = df_raw.shape
attack_cnt = df_raw[df_raw['Label'] != 'BENIGN']['Label'].value_counts().sum()
print('{} attack counts out of {} total counts'.format(attack_cnt,nRow))

shape of raw data:                                (1162213, 79)
missing data(null, NaN, Inf, -Inf):               85
shape of raw data w/o missing values:             (1161344, 79)
shape of raw data w/o single unique values:       (1161344, 71)
shape of raw data w/o duplicates:                 (1017939, 71)
222835 attack counts out of 1017939 total counts


In [8]:
df_raw.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.687500,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,40,0,666666.687500,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,40,2,12,0,0,33,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,1.100917e+05,18348.623047,109.0,0.0,109,109,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,20,20,9174.311523,9174.311523,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,2.307692e+05,38461.539062,52.0,0.0,52,52,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,20,20,19230.769531,19230.769531,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,3.529412e+05,58823.531250,34.0,0.0,34,34,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,20,20,29411.765625,29411.765625,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,1,6,1,6,31,329,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.687500,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,40,0,666666.687500,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,40,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [9]:
# remove unnecessary columns
cols = ['Bwd Packet Length Std','PSH Flag Count','min_seg_size_forward',
        'Min Packet Length','ACK Flag Count','Bwd Packet Length Min',
        'Fwd IAT Std','Init_Win_bytes_forward','Flow IAT Max',
        'Bwd Packets/s','URG Flag Count','Bwd IAT Total','Label']
df = df_raw[cols]
df.head()

,Bwd Packet Length Std,PSH Flag Count,min_seg_size_forward,Min Packet Length,ACK Flag Count,Bwd Packet Length Min,Fwd IAT Std,Init_Win_bytes_forward,Flow IAT Max,Bwd Packets/s,URG Flag Count,Bwd IAT Total,Label
0,0.0,0,20,6,1,0,0.0,33,3,0.000000,0,0,BENIGN
1,0.0,0,20,6,1,6,0.0,29,109,9174.311523,1,0,BENIGN
2,0.0,0,20,6,1,6,0.0,29,52,19230.769531,1,0,BENIGN
3,0.0,0,20,6,1,6,0.0,31,34,29411.765625,1,0,BENIGN
4,0.0,0,20,6,1,0,0.0,32,3,0.000000,0,0,BENIGN


In [10]:
## save to csv file
#df_raw.to_csv('df_preproc_01.csv',index=False)
#df.to_csv('df_preproc_02.csv',index=False)

In [13]:
df = pd.read_csv('df_preproc_02.csv')
# drop features with high correlation (> 0.95)
corr_matrix = df.corr().abs()
upp_matrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
high_corr = [column for column in upp_matrix.columns if any(upp_matrix[column] > 0.95)]
df = df.drop(high_corr, axis=1)
print('shape of raw data w/o high correlation features: ',df.shape)
display(df['Label'].value_counts())

shape of raw data w/o high correlation features:  (1017939, 13)


BENIGN          795104
DDoS            128014
PortScan         90694
Web Attack        2143
Bot               1948
Infiltration        36
Name: Label, dtype: int64

선택된 feature 중 high correlation 발견되지 않음